In [16]:
# Импортируем библиотеку MongoClient из модуля pymongo для работы с MongoDB
from pymongo import MongoClient
import pymongo

# Импортируем класс DuplicateKeyError из модуля pymongo.errors для обработки дубликатов ключей
from pymongo.errors import DuplicateKeyError

# Импортируем модуль json для работы с JSON-данными
import json

# Импортируем библиотеку uuid для генерации уникальных идентификаторов
import uuid

# Импортируем библиотеку pandas для работы с таблицами и данными
import pandas as pd

# Импортируем функцию pprint из модуля pprint для красивого вывода данных
from pprint import pprint


#### Урок 3. Парсинг данных. HTML, Beautiful Soap
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты, то есть цифра вводится одна, а запрос проверяет оба поля)

In [7]:
# Определяем контейнер для датасета
JSON_FILE_PATH = '../Lesson_2/vacancies.json'

**Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии в вашу базу.**


In [25]:
# Подключение к серверу MongoDB
client = MongoClient('127.0.0.1', 27017)

In [26]:
# Доступ к базе данных с именем
db = client['vacancies']

# Доступ к коллекции с именем 'vacancy' в базе данных 'vacancies'
vacancy = db.hh_vacancies

In [29]:
# Функция для добавления вакансий из файла JSON в коллекцию MongoDB
def addVacancy(collection, json_file_path):
    try:
        with open(json_file_path, 'r') as file:
            # Загрузка вакансий из файла JSON
            vacancies_list = json.load(file)
            for el in vacancies_list:
                # Генерация уникального ID для вакансии с использованием uuid
                el['_id'] = str(uuid.uuid4())
                try:
                    # Вставка вакансии в коллекцию
                    collection.insert_one(el)
                    
                    # Создание индекса на поле 'vacancy_id' для ускорения поиска и обеспечения уникальности
                    collection.create_index([('vacancy_id', pymongo.TEXT)], name='search_index', unique=True)
                except pymongo.errors.DuplicateKeyError:
                    print(f"Вакансия с таким id: {el.get('_id')} уже существует")
                    # Обработка ошибок дубликатов ключей
                    pass
                except Exception as e:
                    # Обработка других исключений
                    print(f"Произошла ошибка: {e}")
    except FileNotFoundError:
        print("Файл не найден.")
    except Exception as e:
        print(f"Произошла ошибка при чтении файла: {e}")

# Вызов функции с коллекцией vacancy и путем к файлу JSON
addVacancy(vacancy, JSON_FILE_PATH)

**Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты, то есть цифра вводится одна, а запрос проверяет оба поля)**

In [32]:
# Функция для поиска вакансий с минимальной или максимальной зарплатой выше указанной суммы
def getVacanciesBySalary(amount, collection):
    try:
        # Формируем запрос (query) для поиска вакансий с минимальной или максимальной зарплатой выше указанной суммы
        query = {
            '$or': [
                {'min_salary': {'$gt': amount}},   # Поиск вакансий с минимальной зарплатой выше указанной суммы
                {'max_salary': {'$gt': amount}},   # Поиск вакансий с максимальной зарплатой выше указанной суммы
            ]
        }
        # Выполняем запрос к коллекции MongoDB с использованием метода find()
        # и сортируем результаты по полю 'vacancy_id' в порядке возрастания (1)
        for element in collection.find(query).sort('vacancy_id', 1):
            # Выводим найденные вакансии с помощью функции pprint() для более читаемого вывода
            pprint(element)
    except Exception as e:
        # Обрабатываем исключения, если что-то пошло не так при выполнении запроса
        print(f"Произошла ошибка при чтении файла: {e}")

# Вызов функции с коллекцией vacancy и указанной суммой зарплаты
getVacanciesBySalary(300000, vacancy)

{'_id': 'ade3025d-3227-483e-91e4-9fca6e4685da',
 'currency': 'RUB',
 'employer': 'Росгосстрах',
 'employer_page': 'https://hh.ru/employer/2245?hhtmFrom=vacancy_search_list',
 'experience': 'Опыт от 1 года до 3 лет',
 'job_title': 'Системный аналитик (система финансовой\xa0отчетности)',
 'link': 'https://hh.ru/vacancy/83043654?from=vacancy_search_list&query=%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
 'location': 'Москва, Киевская и еще\xa02\xa0',
 'max_salary': 320000,
 'min_salary': nan,
 'source_site': 'https://hh.ru',
 'vacancy_id': '83043654'}
{'_id': '9997766c-9081-46f1-b16a-50ed51621785',
 'currency': 'RUB',
 'employer': 'ООО\xa0Арте',
 'employer_page': 'https://hh.ru/employer/9187818?hhtmFrom=vacancy_search_list',
 'experience': 'Опыт от 3 до 6 лет',
 'job_title': 'Системный аналитик',
 'link': 'https://hh.ru/vacancy/83227176?from=vacancy_search_list&query=%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA',
 'location': 'Москва',
 'max_salary': 450000,
 'min_salary': 20000

In [23]:
# Закрытие соединения с сервером MongoDB
client.close()